In [2]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor
import gym

import pickle,h5py,json
import urllib
from bs4 import BeautifulSoup

import pygame
from pygame.locals import *

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def calculate_discounted_rewards(R,gamma,T=10,alpha=1):
    disc_rewards = []
    RR = R.copy()
    RR = RR+[0]*(T-1)
    for k0 in range(len(R)):
        RRR = RR[k0:(k0+T)]
        for kk in range(len(RRR)):
            RRR[kk] = alpha*RRR[kk]*(gamma**kk)
        disc_rewards.append(sum(RRR))
    return np.array(disc_rewards)

In [56]:
tf.reset_default_graph()

in0 = tf.placeholder(dtype=tf.float32,shape=(4,1))
out0 = tf.placeholder(dtype=tf.float32,shape=(2,1))
#discrew = tf.placeholder(dtype=tf.float32,shape=(None,1))

w0 = tf.Variable(tf.random_normal([10,4]),name='weights0')
b0 = tf.Variable(tf.random_normal([10,1]),name='biases0')
w1 = tf.Variable(tf.random_normal([2,10]),name='weights1')
b1 = tf.Variable(tf.random_normal([2,1]),name='biases1')

layer0 = tf.nn.relu(tf.matmul(w0,in0)+b0)
softout = tf.nn.softmax(tf.matmul(w1,layer0)+b1,axis=0)

action_sugg = tf.argmax(softout,axis=0)[0]

#log_grad = tf.gradients(tf.log(softout),x)
#delta = tf.reduce_mean(tf.multiply(discrew,log_grad),axis=1)

In [57]:
clock = pygame.time.Clock()
pygame.init()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

env = gym.make('CartPole-v0')
state = env.reset()

S = []
P = []
R = []

game_over = False
while not game_over:
    
    policy = sess.run([softout],feed_dict={
        in0:state.reshape(4,1)
    })
    action_prop = rnd.choice(np.arange(0,2),p=policy[0].flatten())
    
    state,reward,game_over,_ = env.step(action_prop)
    env.render()
    
    S.append(state)
    P.append(policy)
    R.append(reward)
    
    if any([e.type==pygame.KEYDOWN for e in pygame.event.get()]):game_over=True
    clock.tick(5)
    
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [58]:
policy

[array([[0.21049896],
        [0.789501  ]], dtype=float32)]

In [232]:
DR = calculate_discounted_rewards(R,0.99)

In [234]:
lg,d = sess.run([log_grad,delta],feed_dict={
    in0:np.array(S).reshape(-1,4),
    discrew:np.array(DR).reshape(-1,1)
}) 

In [236]:
d

array([[ 0.72496706, -0.6786164 , -0.58989364,  0.15350671,  0.46237272,
        -0.7351186 ,  0.09752943, -0.7211214 , -0.6822029 , -0.31058463]],
      dtype=float32)